# Wie giftig ist der Quelltext?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [ ]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es werden mehrere Abfragen an die Datenbank gestellt. Die Ergebnisse werden in Dataframes (pandas) gespeichert und schließlich in einem Dataframe zusammengeführt.

In [ ]:
import pandas as pd

classfanoutcomplexity_query ="MATCH (c:Type)-[:DEPENDS_ON]->(d:Type) WHERE NOT c.name CONTAINS '$' RETURN DISTINCT c.fqn as Type, count(d) AS ClassFanOutComplexity ORDER BY ClassFanOutComplexity DESC"
classfanoutcomplexity_df = pd.DataFrame(graph.run(classfanoutcomplexity_query).data())

cyclomaticcomplexity_query ="MATCH (c:Type)-[DECLARES]->(m:Method) WHERE NOT c.name CONTAINS '$' AND EXISTS(m.cyclomaticComplexity) RETURN DISTINCT m.signature AS Method, c.fqn AS Type, m.cyclomaticComplexity as CyclomaticComplexity ORDER BY CyclomaticComplexity DESC"
cyclomaticComplexity_df = pd.DataFrame(graph.run(cyclomaticcomplexity_query).data())

methodlength_query ="MATCH (c:Type)-[DECLARES]->(m:Method) WHERE NOT c.name CONTAINS '$' AND EXISTS(m.effectiveLineCount) RETURN DISTINCT m.signature AS Method, c.fqn AS Type, m.effectiveLineCount AS MethodLength ORDER BY MethodLength DESC"
methodLength_df = pd.DataFrame(graph.run(methodlength_query).data())

parameternumber_query ="MATCH (c:Type)-[DECLARES]->(m:Method)-[HAS]->(p:Parameter) WHERE NOT c.name CONTAINS '$' RETURN m.signature AS Method, c.fqn AS Type, count(p) AS ParameterNumber ORDER BY ParameterNumber DESC"
parameterNumber_df = pd.DataFrame(graph.run(parameternumber_query).data())

## 3. Datenaufbereitung
Zunächst werden die vier Dataframes bereinigt. Hierzu müssen alle Zeilen (Klassen oder Methoden) mittels der Methode ```drop()``` und der Option ```inplace = True```gelöscht werden, die geringer als folgende Schwellenwerte sind.

Metrik |	Schwellenwert
---- |----
ClassFanOutComplexity | 30
CyclomaticComplexity | 10
MethodLength | 30
ParameterNumber | 6

In [ ]:
# Entferne alle Klassen, die weniger als 30 Abhängigkeiten zu anderen Klassen besitzen.
# ...

# Entferne alle Methoden, deren zyklomatische Komplexität kleiner als 10 ist.
# ...

# Entferne alle Methoden, deren Anzahl der Quelltextzeilen kleiner als 30 ist.
# ...

# Entferne alle Methoden, die weniger als 6 Paramter besitzen.
# ...

Als Nächstes werden die drei Dataframes ```cyclomaticComplexity_df```, ```methodLength_df``` und ```parameterNumber_df``` mittels der Methode ```merge()``` über die gemeinsame Spalte ```on = 'Type'``` mit der Option ``` how = 'outer'``` zusammengeführt und im Dataframe ```method_metrics_df``` gespeichert. Alle Zellen, die keinen Wert besitzen (NaN), werden mittels der Methode ```fillna(0)``` auf 0 gesetzt. 

In [ ]:
# Die drei Dataframes mit Methodenmetriken werden zusammengeführt und im Dataframe method_metrics_df abgebildet.
# ...

# Fehlende Werte werden auf 0 gesetzt.
# ...

Im folgenden Codeabschnitt werden im Dataframe ```method_metrics_df``` mehrfach auftretende Typen mittels der Methode ```groupBy()``` zusammengeführt.

In [ ]:
# Führe mehrfach auftretende Typen zusammen.
# ...

Als Nächstes werden die beiden Dataframes ```classfanoutcomplexity_df``` und ```method_metrics_df``` mittels der Methode ```merge()``` über die gemeinsame Spalte ```on = 'Type'``` mit der Option ``` how = 'outer'``` zusammengeführt und im Dataframe ```toxicity_df``` gespeichert. Alle Zellen, die keinen Wert besitzen (NaN), werden mittels der Methode ```fillna(0)``` auf 0 gesetzt. 

In [ ]:
# Die Dataframes classfanoutcomplexity_df und method_metrics_df werden zusammengeführt.
# ...

# Fehlende Werte werden auf 0 gesetzt.
# ...

Jetzt wird mittels ```apply()``` im resultierenden Dataframe für jede Spalte der Toxicity-Wert berechnet, indem die absoluten Werte durch den jeweiligen Schwellenwert geteilt werden. Der absolute Wert wird dabei stets überschrieben. Zudem wird die Spalte ```ToxicityScore``` als Summe aller Toxicity-Werte für eine Klasse eingeführt und alle Werte entsprechend des Toxicity-Scores absteigend sortiert.

In [ ]:
# Berechne den Toxicity-Wert für ClassFanOutComplexity.
# ...

# Berechne den Toxicity-Wert für CyclomaticComplexity.
# ...

# Berechne den Toxicity-Wert für MethodLength.
# ...

# Berechne den Toxicity-Wert für ParameterNumber.
# ...

# Erzeuge die Spalte ToxicityScore und berechne die Zeilensumme aus ClassFanOutComplexity, CyclomaticComplexity, MethodLength und ParameterNumber.
# ...

# Sortiere alle Werte absteigend nach dem ToxicityScore.
# ...

## 4. Visualisierung
Die Daten werden mittels eines Stacked Barcharts visualisiert.



In [ ]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [ ]:
import pygal
stacked_bar_chart = pygal.StackedBar(show_legend=True, human_readable=True, fill=False, x_label_rotation=90, truncate_label=-1, truncate_legend=-1)
stacked_bar_chart.title = 'Toxicity Chart'
stacked_bar_chart.x_labels = toxicity_df['Type'].tolist()
stacked_bar_chart.add('ClassFanOutComplexity', toxicity_df['ClassFanOutComplexity'].tolist())
stacked_bar_chart.add('CyclomaticComplexity', toxicity_df['CyclomaticComplexity'].tolist())
stacked_bar_chart.add('MethodLength', toxicity_df['MethodLength'].tolist())
stacked_bar_chart.add('ParameterNumber', toxicity_df['ParameterNumber'].tolist())
display(HTML(base_html.format(rendered_chart=stacked_bar_chart.render(is_unicode=True))))